This notebook implements the <b>TextFooler</b> attack on the SQuAD (Stanford Question Answering Dataset) 2.0 dataset.<br /><br />

The main purpose of this notebook is to create adversarial examples for question-answering systems using the TextFooler method. It replaces words with synonyms while preserving the overall structure and meaning of the text. The attack targets words based on their part of speech, avoiding modification of certain grammatical elements to maintain readability.<br /><br />

The generated adversarial examples are designed to challenge question-answering models by introducing subtle word-level changes that may affect the model's performance without significantly altering the human-readable content. This approach allows for testing the robustness of QA models against lexical variations while keeping the core information intact.<br /><br />

The script processes the entire SQuAD dataset, creating a new version with adversarial examples that can be used for evaluating and potentially improving the resilience of question-answering systems against this type of word-level perturbation.

### Loading libraries

Imports necessary libraries for natural language processing (spaCy, NLTK), data manipulation (json), and progress tracking (tqdm).

In [1]:
import os
import json
import random
import spacy
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
from difflib import SequenceMatcher

### NLP model loading
It loads the spaCy English model for part-of-speech tagging and tokenization.

In [2]:
# Load spaCy model
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [3]:
# Download WordNet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ferhatsarikaya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Synonym generation
get_synonyms() uses NLTK's WordNet to generate synonyms for a given word.

In [4]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)


### TextFooler attack implementation
textfooler_attack() is the core function that:
<ul>
    <li>Tokenizes the input text and gets part-of-speech tags.</li>
    <li>Preserves specified phrases from modification.</li>
    <li>Replaces words with synonyms based on a random threshold, excluding certain parts of speech (punctuation, determiners, adpositions, conjunctions).</li>
</ul>

In [5]:
def textfooler_attack(text, importance_threshold=0.5, preserve_phrases=None):
    if preserve_phrases is None:
        preserve_phrases = set()
    else:
        preserve_phrases = set(phrase.lower() for phrase in preserve_phrases)
    
    doc = nlp(text)
    words = [token.text for token in doc]
    pos_tags = [token.pos_ for token in doc]
    
    # Check for preserved phrases
    for i in range(len(words)):
        for j in range(i+1, len(words)+1):
            if ' '.join(words[i:j]).lower() in preserve_phrases:
                for k in range(i, j):
                    pos_tags[k] = 'PRESERVED'
    
    for i, (word, pos) in enumerate(zip(words, pos_tags)):
        if pos not in ['PRESERVED', 'PUNCT', 'DET', 'ADP', 'CCONJ', 'SCONJ'] and random.random() > importance_threshold:
            synonyms = get_synonyms(word)
            if synonyms:
                words[i] = random.choice(synonyms)
    
    return ' '.join(words)


### Answer position finding
find_answer_position() locates the position of an answer within a modified context using fuzzy matching.

In [6]:
def find_answer_position(context, answer):
    best_match = None
    best_ratio = 0
    words = context.split()
    answer_words = answer.split()
    
    for i in range(len(words) - len(answer_words) + 1):
        substring = ' '.join(words[i:i+len(answer_words)])
        ratio = SequenceMatcher(None, substring.lower(), answer.lower()).ratio()
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = ' '.join(words[:i])
    
    return len(best_match) + 1 if best_ratio > 0.8 else -1


### Dataset processing
process_squad_file() applies the TextFooler attack to the SQuAD dataset by:
<ul>
    <li>Modifying both context paragraphs and questions.</li>
    <li>Preserving answer phrases in the context.</li>
    <li>Updating answer positions in modified contexts.</li>
</ul>

In [7]:
def process_squad_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    new_data = {"version": "v2.0", "data": []}

    for article in tqdm(data['data'], desc="Processing articles"):
        new_article = {"title": article['title'], "paragraphs": []}
        
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            
            # Collect all answers to preserve
            answers = set()
            for qa in paragraph['qas']:
                if qa['answers']:
                    answers.add(qa['answers'][0]['text'])
            
            # Apply TextFooler to the context, preserving answers
            adv_context = textfooler_attack(context, preserve_phrases=answers)
            
            new_qas = []
            for qa in paragraph['qas']:
                new_qa = qa.copy()
                
                # Apply TextFooler to the question, preserving the answer
                if qa['answers']:
                    answer = qa['answers'][0]['text']
                    new_qa['question'] = textfooler_attack(qa['question'], preserve_phrases=[answer])
                else:
                    new_qa['question'] = textfooler_attack(qa['question'])
                
                if new_qa['answers']:
                    answer = new_qa['answers'][0]['text']
                    answer_start = find_answer_position(adv_context, answer)
                    
                    if answer_start != -1:
                        new_qa['answers'][0]['answer_start'] = answer_start
                    else:
                        print(f"Warning: Answer '{answer}' not found in modified context. Keeping original position.")
                        # Keep the original answer_start
                
                new_qas.append(new_qa)
            
            new_paragraph = {
                "context": adv_context,
                "qas": new_qas
            }
            new_article['paragraphs'].append(new_paragraph)
        
        new_data['data'].append(new_article)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(new_data, f, ensure_ascii=False, indent=2)

### Main execution
The script applies the TextFooler attack to the SQuAD v2.0 training set and generates a new file with adversarial examples.

In [8]:
# Create data set
path = "SQuAD/"
input_file = os.path.join(path, "train-v2.0.json")
output_file = os.path.join(path, "squad-v2.0-textfooler.json")
process_squad_file(input_file, output_file)

Processing articles:   0%|                              | 0/442 [00:00<?, ?it/s]

Processing articles:   0%|                      | 1/442 [00:07<54:23,  7.40s/it]

Processing articles:   0%|                      | 2/442 [00:12<44:30,  6.07s/it]

Processing articles:   1%|▏                     | 3/442 [00:16<38:27,  5.26s/it]

Processing articles:   1%|▏                     | 4/442 [00:18<28:35,  3.92s/it]

...

Processing articles:  99%|███████████████████▊| 437/442 [15:11<00:09,  1.84s/it]

Processing articles:  99%|███████████████████▊| 438/442 [15:13<00:07,  1.87s/it]

Processing articles:  99%|███████████████████▊| 439/442 [15:16<00:06,  2.09s/it]

Processing articles: 100%|███████████████████▉| 440/442 [15:18<00:04,  2.03s/it]

Processing articles: 100%|████████████████████| 442/442 [15:19<00:00,  2.08s/it]
